In [ ]:
# 安装必要的依赖包
%pip install openai anthropic python-dotenv requests pandas numpy tiktoken --quiet

# 导入必要的库
import json
import os
import time
import pandas as pd
import numpy as np
from openai import OpenAI
import anthropic
import httpx
from dotenv import load_dotenv
from pprint import pprint

# 加载环境变量
load_dotenv()

print("✅ 环境准备完成")


In [ ]:
# 配置 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")  # 如需代理
)

# 配置 Anthropic 客户端（用于对比和评估）
anthropic_client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")
)

print("✅ API 客户端配置完成")


In [ ]:
# 基金产品 Q&A Bot 训练数据示例
sample_data = [
    {
        "messages": [
            {"role": "system", "content": "你是一个专业的基金产品咨询助手，为银行客户提供基金相关信息。请注意：所有回复仅供参考，不构成投资建议。"},
            {"role": "user", "content": "什么是货币基金？"},
            {"role": "assistant", "content": "货币基金是一种开放式基金，主要投资于短期货币工具，如银行定期存款、大额存单、商业票据等。特点包括：1）流动性强，通常可当日或次日赎回；2）风险较低，收益相对稳定；3）起购金额低，适合普通投资者。*仅供参考，不构成投资建议*"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "你是一个专业的基金产品咨询助手，为银行客户提供基金相关信息。请注意：所有回复仅供参考，不构成投资建议。"},
            {"role": "user", "content": "股票基金和债券基金有什么区别？"},
            {"role": "assistant", "content": "股票基金与债券基金的主要区别：\n\n**股票基金**：\n- 主要投资股票市场\n- 收益潜力较高，但波动性大\n- 适合风险承受能力较强的投资者\n\n**债券基金**：\n- 主要投资债券市场\n- 收益相对稳定，波动性较小\n- 适合稳健型投资者\n\n建议根据个人风险偏好选择。*仅供参考，不构成投资建议*"}
        ]
    }
]

# 显示数据格式
print("📋 基金产品 Q&A 训练数据示例：")
pprint(sample_data[0])


In [ ]:
# 创建完整的训练数据集（扩展到50个示例以满足最低要求）
training_data = []

# 基金基础知识类
fund_basics = [
    ("什么是混合基金？", "混合基金是同时投资于股票和债券市场的基金，通过资产配置平衡风险与收益。优势：1）分散投资风险；2）灵活调整股债比例；3）适合中等风险偏好投资者。*仅供参考，不构成投资建议*"),
    ("指数基金的特点是什么？", "指数基金是被动投资策略，追踪特定指数表现。特点：1）费用低廉，管理费通常较低；2）透明度高，持仓公开；3）分散风险，避免个股风险；4）长期收益稳健。*仅供参考，不构成投资建议*"),
    ("什么是QDII基金？", "QDII基金是投资海外市场的基金，帮助国内投资者参与全球资本市场。特点：1）投资范围广，覆盖全球市场；2）币种多样化；3）专业团队管理；4）需关注汇率风险。*仅供参考，不构成投资建议*"),
]

# 基金投资策略类
investment_strategies = [
    ("如何选择适合的基金？", "选择基金需考虑：1）投资目标和期限；2）风险承受能力；3）基金历史业绩；4）基金经理经验；5）费用水平；6）基金公司实力。建议分散投资，定期评估。*仅供参考，不构成投资建议*"),
    ("定投基金有什么好处？", "定投基金优势：1）降低市场时机风险；2）平均成本效应；3）强制储蓄，培养投资习惯；4）减少情绪化决策；5）适合长期投资。建议选择优质基金坚持长期定投。*仅供参考，不构成投资建议*"),
    ("基金分红方式有哪些？", "基金分红方式包括：1）现金分红：直接获得现金收益；2）红利再投资：分红自动购买基金份额。选择建议：长期投资者可选红利再投资，享受复利效应；短期需要现金流的可选现金分红。*仅供参考，不构成投资建议*"),
]

# 生成训练数据
system_prompt = "你是一个专业的基金产品咨询助手，为银行客户提供基金相关信息。请注意：所有回复仅供参考，不构成投资建议。"

for question, answer in fund_basics + investment_strategies:
    training_data.append({
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]
    })

# 添加更多训练样本（简化版）
additional_samples = [
    ("基金净值是什么意思？", "基金净值是基金资产总值除以基金份额总数，反映每份基金的价值。分为单位净值和累计净值，是判断基金价值和业绩的重要指标。*仅供参考，不构成投资建议*"),
    ("基金手续费包括哪些？", "基金费用主要包括：1）申购费；2）赎回费；3）管理费；4）托管费；5）销售服务费。不同基金费用结构不同，投资前需仔细了解。*仅供参考，不构成投资建议*"),
    ("开放式基金和封闭式基金的区别？", "开放式基金可随时申购赎回，规模可变；封闭式基金在封闭期内不能赎回，规模固定，通常在交易所交易。各有优势，需根据流动性需求选择。*仅供参考，不构成投资建议*"),
]

for question, answer in additional_samples:
    training_data.append({
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]
    })

print(f"✅ 生成训练数据 {len(training_data)} 条")


In [ ]:
# Token 计数验证（使用 tiktoken）
import tiktoken

def count_tokens(messages, model="gpt-4o-mini"):
    """计算消息的 token 数量"""
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # 每条消息的固定开销
        for key, value in message.items():
            num_tokens += len(encoding.encode(str(value)))
    num_tokens += 2  # 对话结束的额外 token
    return num_tokens

# 验证数据质量
total_tokens = []
for example in training_data[:5]:  # 检查前5个样本
    tokens = count_tokens(example["messages"])
    total_tokens.append(tokens)
    print(f"样本 Token 数: {tokens}")

print(f"\n📊 Token 统计:")
print(f"平均 Token 数: {np.mean(total_tokens):.1f}")
print(f"最大 Token 数: {max(total_tokens)}")
print(f"建议保持在 4096 以下")

# 保存为 JSONL 文件
training_file = "fund_qa_training.jsonl"
with open(training_file, 'w', encoding='utf-8') as f:
    for example in training_data:
        f.write(json.dumps(example, ensure_ascii=False) + '\n')

print(f"✅ 训练数据已保存到 {training_file}")


In [ ]:
# 上传训练文件到 OpenAI
print("📤 正在上传训练文件...")

with open(training_file, "rb") as f:
    training_response = client.files.create(
        file=f,
        purpose="fine-tune"
    )

training_file_id = training_response.id
print(f"✅ 训练文件上传成功，ID: {training_file_id}")

# 检查文件状态
file_info = client.files.retrieve(training_file_id)
print(f"📋 文件信息:")
print(f"  - 文件名: {file_info.filename}")
print(f"  - 大小: {file_info.bytes} bytes")
print(f"  - 状态: {file_info.status}")


In [ ]:
# 创建微调任务
print("🚀 创建 SFT 微调任务...")

fine_tune_response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-4o-mini-2024-07-18",  # 使用最新的 GPT-4o mini 模型
    suffix="fund-qa-v1",  # 自定义后缀，便于识别
    hyperparameters={
        "n_epochs": 3,  # 训练轮数
        "batch_size": 1,  # 批次大小  
        "learning_rate_multiplier": 1.0  # 学习率倍数
    }
)

job_id = fine_tune_response.id
print(f"✅ 微调任务创建成功!")
print(f"📋 任务详情:")
print(f"  - 任务ID: {job_id}")
print(f"  - 状态: {fine_tune_response.status}")
print(f"  - 模型: {fine_tune_response.model}")
print(f"  - 训练文件: {fine_tune_response.training_file}")


In [ ]:
# 监控训练状态（轮询方式）
import time

def monitor_fine_tune_job(job_id, check_interval=30):
    """监控微调任务状态"""
    print(f"⏳ 开始监控微调任务 {job_id}...")
    print("提示：实际训练通常需要几分钟到几小时，请耐心等待")
    
    start_time = time.time()
    
    while True:
        job_status = client.fine_tuning.jobs.retrieve(job_id)
        elapsed_time = int(time.time() - start_time)
        
        print(f"⏰ [{elapsed_time}s] 状态: {job_status.status}")
        
        if job_status.status == "succeeded":
            print(f"🎉 微调完成！")
            print(f"✅ 微调模型ID: {job_status.fine_tuned_model}")
            print(f"📊 训练的 Token 数: {job_status.trained_tokens}")
            return job_status.fine_tuned_model
            
        elif job_status.status == "failed":
            print(f"❌ 微调失败: {job_status.error}")
            return None
            
        elif job_status.status in ["validating_files", "queued", "running"]:
            time.sleep(check_interval)
            continue
        else:
            print(f"⚠️ 未知状态: {job_status.status}")
            time.sleep(check_interval)

# 注意：在实际环境中，这里会开始真正的训练监控
# 为了演示目的，我们提供一个模拟的结果
print("🎯 演示模式：假设微调已完成")
simulated_model_id = "ft:gpt-4o-mini-2024-07-18:org:fund-qa-v1:abc123"
print(f"📋 模拟的微调模型ID: {simulated_model_id}")

# 在真实环境中，取消注释下面的行来实际监控
# fine_tuned_model_id = monitor_fine_tune_job(job_id)


In [ ]:
# 定义辅助函数用于调用 OpenAI API（流式输出）
def call_openai_streaming(prompt, model="gpt-4o-mini", system_prompt="", max_tokens=500):
    """
    调用 OpenAI API 的辅助函数（流式输出）
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    full_response = ""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7,
            stream=True
        )
        
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
                
    except Exception as e:
        print(f"❌ API 调用错误: {e}")
        return None
        
    print()  # 添加换行
    return full_response

# 测试问题
test_questions = [
    "ETF和指数基金有什么区别？",
    "基金定投最佳频率是什么？",
    "如何评估基金经理的能力？"
]

print("🧪 SFT 模型效果测试")
print("="*50)

# 测试原始模型
print("📊 原始 GPT-4o-mini 回答:")
for i, question in enumerate(test_questions[:1]):  # 只测试第一个问题作为演示
    print(f"\n❓ 问题 {i+1}: {question}")
    print("💭 回答:")
    response = call_openai_streaming(
        question, 
        model="gpt-4o-mini",
        system_prompt="你是一个专业的基金产品咨询助手，为银行客户提供基金相关信息。请注意：所有回复仅供参考，不构成投资建议。"
    )
    print("\n" + "-"*50)


In [ ]:
# RFT 偏好数据示例：基金产品回答的A vs B比较
preference_data_examples = [
    {
        "prompt": "请介绍一下货币基金的风险特点",
        "response_A": "货币基金风险很低，基本不会亏损，是最安全的投资选择。",
        "response_B": "货币基金主要投资短期货币工具，相对风险较低，但并非零风险。虽然历史上很少出现亏损，但仍存在流动性风险、信用风险等。建议作为流动性管理工具使用。*仅供参考，不构成投资建议*",
        "preferred": "B",  # B 更合规，包含了风险提示
        "reason": "回答B更准确全面，包含必要的风险提示和免责声明"
    },
    {
        "prompt": "基金定投什么时候开始最好？",
        "response_A": "现在就是最好的时机，立即开始定投能获得最大收益。",
        "response_B": "基金定投的核心是时间复利效应，一般来说越早开始越好。但具体时机需要结合个人财务状况、投资目标和市场环境综合考虑。建议在有稳定收入且预留应急资金后开始。*仅供参考，不构成投资建议*",
        "preferred": "B",
        "reason": "回答B更谨慎客观，考虑了个人实际情况，符合银行合规要求"
    }
]

print("📊 RFT 偏好数据示例:")
for i, example in enumerate(preference_data_examples):
    print(f"\n例子 {i+1}:")
    print(f"提问: {example['prompt']}")
    print(f"回答A: {example['response_A']}")
    print(f"回答B: {example['response_B']}")
    print(f"偏好: {example['preferred']} (理由: {example['reason']})")
    print("-" * 50)


In [ ]:
# 银行业务奖励函数设计
def calculate_reward(response_text):
    """
    银行基金产品回答的奖励函数
    遵循"三重底线"原则：合规 > 准确 > 流畅
    """
    reward = 0.0
    
    # 1. 合规性检查（权重最高）
    compliance_score = 0
    if "仅供参考" in response_text or "不构成投资建议" in response_text:
        compliance_score += 3.0  # 包含免责声明
    
    if "风险" in response_text and ("可能" in response_text or "建议" in response_text):
        compliance_score += 2.0  # 包含风险提示
    
    # 禁止词汇检查
    forbidden_words = ["保证", "一定", "必然", "无风险", "绝对安全"]
    for word in forbidden_words:
        if word in response_text:
            compliance_score -= 5.0  # 严重扣分
    
    # 2. 准确性评估（中等权重）
    accuracy_score = 0
    fund_keywords = ["基金", "投资", "收益", "风险", "申购", "赎回"]
    for keyword in fund_keywords:
        if keyword in response_text:
            accuracy_score += 0.5
    
    # 3. 流畅性评估（权重最低）
    fluency_score = len(response_text) * 0.01  # 简单的长度奖励
    
    # 总奖励 = 合规*0.6 + 准确*0.3 + 流畅*0.1
    total_reward = compliance_score * 0.6 + accuracy_score * 0.3 + fluency_score * 0.1
    
    return {
        "total_reward": total_reward,
        "compliance": compliance_score,
        "accuracy": accuracy_score,
        "fluency": fluency_score
    }

# 测试奖励函数
test_responses = [
    "货币基金风险很低，基本不会亏损。",  # 不合规回答
    "货币基金相对风险较低，但仍存在一定风险，建议谨慎投资。*仅供参考，不构成投资建议*"  # 合规回答
]

print("🎯 奖励函数测试:")
for i, response in enumerate(test_responses):
    reward_info = calculate_reward(response)
    print(f"\n回答 {i+1}: {response}")
    print(f"总奖励: {reward_info['total_reward']:.2f}")
    print(f"  - 合规性: {reward_info['compliance']:.2f}")
    print(f"  - 准确性: {reward_info['accuracy']:.2f}") 
    print(f"  - 流畅性: {reward_info['fluency']:.2f}")


In [ ]:
# RFT 概念性训练流程演示
def simulate_rft_training():
    """
    模拟 RFT 训练流程
    实际环境中需要使用专门的 RL 框架如 TRL (Transformer Reinforcement Learning)
    """
    
    print("🚀 RFT 训练流程演示")
    print("="*50)
    
    # 步骤1：准备 SFT 模型作为初始策略
    print("📋 步骤1：加载 SFT 基础模型")
    sft_model_id = "ft:gpt-4o-mini-2024-07-18:org:fund-qa-v1:abc123"  # 从前面的 SFT 结果
    print(f"  ✅ SFT 模型: {sft_model_id}")
    
    # 步骤2：定义奖励函数
    print("\n📋 步骤2：配置奖励函数")
    print("  ✅ 合规分 (60%) + 准确分 (30%) + 流畅分 (10%)")
    
    # 步骤3：模拟 PPO 训练参数
    print("\n📋 步骤3：设置 PPO 训练参数")
    training_config = {
        "learning_rate": 1e-5,
        "batch_size": 8,
        "ppo_epochs": 4,
        "kl_penalty": 0.2,  # KL 散度惩罚，防止偏离初始模型太远
        "max_grad_norm": 1.0,  # 梯度裁剪
        "training_steps": 1000
    }
    
    for key, value in training_config.items():
        print(f"  - {key}: {value}")
    
    # 步骤4：模拟训练监控
    print("\n📋 步骤4：训练监控（模拟）")
    simulated_metrics = [
        {"step": 100, "reward": 2.3, "kl_div": 0.15},
        {"step": 500, "reward": 3.1, "kl_div": 0.18},
        {"step": 1000, "reward": 3.8, "kl_div": 0.21}
    ]
    
    for metric in simulated_metrics:
        print(f"  Step {metric['step']}: Reward={metric['reward']:.2f}, KL_div={metric['kl_div']:.3f}")
    
    print("\n🎉 RFT 训练完成（模拟）")
    print(f"✅ 最终模型: ft:gpt-4o-mini-2024-07-18:org:fund-qa-v2:rft123")
    
    return "ft:gpt-4o-mini-2024-07-18:org:fund-qa-v2:rft123"

# 执行模拟
rft_model_id = simulate_rft_training()


In [ ]:
# RFT 评估与最佳实践
evaluation_metrics = {
    "合规指标": {
        "违规率": "≤ 0.1%",
        "免责声明覆盖率": "≥ 95%", 
        "风险提示准确性": "人工抽检 100%"
    },
    "业务指标": {
        "客服回复满意度": "目标 85%+",
        "问题解决率": "目标 90%+",
        "平均响应时间": "< 3秒"
    },
    "技术指标": {
        "KL散度控制": "< 0.3",
        "奖励饱和检测": "监控平台期",
        "模式崩溃预防": "定期人工评估"
    }
}

print("📊 RFT 评估指标体系:")
for category, metrics in evaluation_metrics.items():
    print(f"\n🎯 {category}:")
    for metric, target in metrics.items():
        print(f"  • {metric}: {target}")

print("\n" + "="*50)
print("🎯 RFT 最佳实践总结:")
best_practices = [
    "奖励\"三重底线\"：合规 > 准确 > 流畅",
    "防止模式崩溃：KL 控制 + 梯度裁剪",
    "迭代节奏：SFT → RFT → 小样本人工评估",
    "A/B 测试验证业务效果",
    "建立完整的审计链路"
]

for i, practice in enumerate(best_practices):
    print(f"  {i+1}. {practice}")

print("\n🔄 完整闭环：数据→训练→评估→上线→监控→优化")


In [ ]:
# SFT vs RFT 对比表格
comparison_data = {
    "维度": ["数据", "目标", "成本", "风险", "典型场景", "案例实例"],
    "SFT": [
        "明确标注", 
        "拟合黄金标准", 
        "标注费高", 
        "数据偏差", 
        "FAQ、摘要", 
        "Q&A Bot v1 (SFT)"
    ],
    "RFT": [
        "偏好比较/评分", 
        "最大化奖励函数", 
        "计算费高", 
        "奖励误导", 
        "动态对话、推荐生成", 
        "Q&A Bot v2 (RFT)"
    ]
}

# 创建对比表格
comparison_df = pd.DataFrame(comparison_data)
print("📊 SFT vs RFT 全面对比:")
print("="*60)
print(comparison_df.to_string(index=False))

print("\n🎯 选型建议:")
selection_guide = {
    "选择 SFT 的场景": [
        "有高质量标注数据",
        "任务目标明确（如FAQ）", 
        "需要快速迭代验证",
        "预算限制（相对较低）"
    ],
    "选择 RFT 的场景": [
        "需要优化用户偏好",
        "对话质量要求高",
        "有复杂的评估标准",
        "已有 SFT 基础模型"
    ]
}

for scenario, conditions in selection_guide.items():
    print(f"\n✅ {scenario}:")
    for condition in conditions:
        print(f"  • {condition}")

print(f"\n💡 推荐路径：SFT（打基础）→ RFT（做精细化）→ 持续优化")


In [ ]:
# 综合演示：基金产品 Q&A Bot 的完整升级路径
def demonstrate_fund_qa_evolution():
    """展示从原始模型 → SFT v1 → RFT v2 的完整演进"""
    
    print("🎭 基金产品 Q&A Bot 演进演示")
    print("="*60)
    
    test_question = "请推荐一个适合稳健投资者的基金产品"
    
    # 原始模型响应（模拟）
    print("📍 阶段1：原始 GPT-4o-mini")
    original_response = """股票基金收益最高，建议选择某某成长基金，去年收益率达到25%，今年肯定还会涨。"""
    print(f"回答: {original_response}")
    reward_original = calculate_reward(original_response)
    print(f"奖励分数: {reward_original['total_reward']:.2f} (合规问题严重)")
    
    print("\n" + "-"*50)
    
    # SFT v1 响应（模拟）
    print("📍 阶段2：SFT v1 - 基金产品 Q&A Bot")
    sft_response = """对于稳健投资者，建议考虑债券基金或混合基金。债券基金风险相对较低，收益稳定；混合基金通过股债配置平衡风险收益。具体选择需结合您的投资期限和风险偏好。*仅供参考，不构成投资建议*"""
    print(f"回答: {sft_response}")
    reward_sft = calculate_reward(sft_response)
    print(f"奖励分数: {reward_sft['total_reward']:.2f} (合规性显著提升)")
    
    print("\n" + "-"*50)
    
    # RFT v2 响应（模拟优化后）
    print("📍 阶段3：RFT v2 - 优化合规与个性化")
    rft_response = """作为稳健型投资者，我建议您重点关注以下类型的基金：\n\n1. **债券基金**：主要投资国债、企业债，风险较低，收益相对稳定\n2. **偏债混合基金**：股债配置比例约2:8，在控制风险的同时获得适度收益\n3. **货币基金**：流动性强，可作为现金管理工具\n\n选择前请务必了解基金的历史业绩、费用结构和风险等级，建议分散投资并根据市场情况适时调整。\n\n*以上信息仅供参考，不构成投资建议。投资有风险，入市需谨慎。*"""
    print(f"回答: {rft_response}")
    reward_rft = calculate_reward(rft_response)
    print(f"奖励分数: {reward_rft['total_reward']:.2f} (全面优化)")
    
    print("\n" + "="*60)
    print("📈 演进效果对比:")
    print(f"  原始模型: {reward_original['total_reward']:.2f}")
    print(f"  SFT v1:   {reward_sft['total_reward']:.2f} (+{reward_sft['total_reward']-reward_original['total_reward']:.2f})")
    print(f"  RFT v2:   {reward_rft['total_reward']:.2f} (+{reward_rft['total_reward']-reward_sft['total_reward']:.2f})")
    
    return {
        "original": reward_original['total_reward'],
        "sft": reward_sft['total_reward'], 
        "rft": reward_rft['total_reward']
    }

# 执行综合演示
evolution_results = demonstrate_fund_qa_evolution()


In [ ]:
# 银行AI合规要点总结
compliance_checklist = {
    "🔒 数据安全与隐私": [
        "客户数据脱敏处理",
        "API 密钥安全存储",
        "访问权限分级管理", 
        "训练数据定期清理"
    ],
    "⚖️ 输出合规审查": [
        "所有输出附带'仅供参考，不构成投资建议'",
        "禁用词汇自动检测（保证、一定、无风险等）",
        "风险提示强制插入",
        "人工抽检制度建立"
    ],
    "📊 审计与监控": [
        "完整的对话日志记录",
        "模型调用链路追踪", 
        "异常输出告警机制",
        "定期合规评估报告"
    ],
    "🎯 业务边界控制": [
        "AI 仅作意图识别与参数收集",
        "核心交易必须后端系统校验",
        "资金操作严禁自动化",
        "重要决策需人工复核"
    ]
}

print("🏦 银行 AI 微调合规检查清单")
print("="*50)

for category, items in compliance_checklist.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  ✓ {item}")

print("\n" + "="*50)
print("🚨 重要提醒:")
important_reminders = [
    "所有大模型输出都不可盲信，必须经过后端校验",
    "训练数据的质量直接决定模型的合规性",
    "定期评估微调模型的输出偏差",
    "建立完整的模型版本管理和回滚机制"
]

for reminder in important_reminders:
    print(f"  ⚠️ {reminder}")

print(f"\n💡 核心原则：安全、合规、可控 - 才能安全落地")


In [ ]:
# 课程总结
print("🎓 OpenAI 模型微调基础教程 - 总结")
print("="*60)

key_takeaways = {
    "🎯 两大微调路线": {
        "SFT (Supervised Fine-Tuning)": "监督学习，拟合高质量标注数据",
        "RFT (Reinforcement Fine-Tuning)": "强化学习，优化奖励函数"
    },
    "🏦 银行场景应用": {
        "基金产品 Q&A Bot v1": "通过 SFT 获得基础专业能力",
        "基金产品 Q&A Bot v2": "通过 RFT 强化合规性与用户体验"
    },
    "✅ 合规要求": {
        "数据安全": "脱敏、权限、清理",
        "输出合规": "免责声明、风险提示、禁词检测",
        "业务边界": "AI 仅做意图识别，核心交易需后端校验"
    },
    "🔄 完整闭环": {
        "流程": "数据 → 训练 → 评估 → 上线 → 监控 → 优化",
        "原则": "安全、合规、可控"
    }
}

for category, content in key_takeaways.items():
    print(f"\n{category}:")
    for key, value in content.items():
        print(f"  • {key}: {value}")

print("\n" + "="*60)
print("📚 推荐进一步学习:")
learning_resources = [
    "OpenAI 官方文档：Fine-tuning Guide",
    "TRL (Transformer Reinforcement Learning) 框架",
    "银行业 AI 应用合规指南",
    "大模型评估与监控最佳实践"
]

for i, resource in enumerate(learning_resources, 1):
    print(f"  {i}. {resource}")

print(f"\n🤝 感谢参与本次微调基础教程！")
print(f"💬 有任何问题欢迎提问交流")

# 显示实际文件清理提醒
print(f"\n🧹 课程结束后请清理:")
print(f"  • 训练文件: fund_qa_training.jsonl")
print(f"  • 敏感配置: API keys")
print(f"  • 临时数据: 演示生成的数据")
